#Итоговый проект

In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from os import listdir
import codecs
import pickle                                                                   
from bs4 import BeautifulSoup as BS
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
%matplotlib inline

In [5]:
def save_obj(obj, name ):                                                       
    with open(name + '.pkl', 'wb') as f:                                        
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)                            
                                                                                
def load_obj(name ):                                                            
    with open(name + '.pkl', 'rb') as f:                                        
        return pickle.load(f)       

In [11]:
path = 'content/'
texts = {}
for filename in tqdm(listdir(path)):
    doc_id = int(filename.strip('.dat'))
    if doc_id not in train_data.doc_id.values:
        continue
    with codecs.open(path + filename, 'r', 'utf-8') as f:
        url = f.readline().strip()
        html = f.read()
        bs = BS(html, 'html.parser')#ищем след страницу
        for s in bs.select('script'):
            s.extract()
        #title = bs.find_all('p')
        #title = list(map(lambda x: x.get_text(), title))
        #print(title)
        s = bs.get_text()
     #   s.translate(string.maketrans("", "", ), bad_chars)
        s=s.replace('\n',"")
        s=s.replace('\t',"")
        s=s.replace('\r',"")
        texts[doc_id] = s
        #train_data.loc[train_data.doc_id == doc_id,'text'] = s
#        train_df = train_df.append({'doc_id':int(filename.strip('.dat')),'text':s},ignore_index=True)
#        print(bs.get_text())

In [12]:
save_obj(texts,'train_data_text')
len(texts)

11648

In [8]:
train_data = pd.read_csv('train_groups.csv',dtype=np.int16)

In [6]:
texts = load_obj('train_data_text')

In [9]:
train_data['text'] = ''

In [10]:
train_data['text'] = train_data.apply(lambda row: texts[row.doc_id], axis = 1)

In [11]:
train_data

,pair_id,group_id,doc_id,target,text
0,1,1,15731,0,ВАЗ 21213 | Замена подшипников ступицы | Нива ...
1,2,1,14829,0,"Ваз 2107 оптом в Сочи. Сравнить цены, купить ..."
2,3,1,15764,0,Купить ступица Лада калина2. Трансмиссия - пер...
3,4,1,17669,0,Классика 21010 - 21074ГЛАВНАЯПродукцияИнтернет...
4,5,1,14852,0,Ступица Нива — замена подшипника своими руками...
...,...,...,...,...,...
11685,11686,129,26672,0,❤★✿★АПРЕЛЯТА 2014 -6❤★✿ ...
11686,11687,129,25838,0,:: Gästebuch #blockStyle404Main64 {text-ali...
11687,11688,129,25703,0,Jizolofej: ArchiveJizolofejMonth Filter by pos...
11688,11689,129,27885,0,Как зовут парня дианы шурыгиной | Пусть говоря...


In [8]:
train_data.shape

(11690, 5)

In [9]:
train_data[train_data.doc_id==4062]

,pair_id,group_id,doc_id,target,text
3936,3937,44,4062,1,/*** vBulletin 3.8.9 CSS* Style: 'О...


In [12]:
save_obj(train_data,'train_data')

In [ ]:
#START HEAR

In [ ]:
train_data = load_obj('train_data')

In [13]:
def get_splits(data,n=3):
    bound =[]
    max_group = data.group_id.max()
    bound.extend([i*int(max_group/n)+1 for i in range(1,n)])
    bound.append(max_group+1)
    prev_id = 1
    for group_id in bound:
        yield (data[~((prev_id <= data.group_id) & (data.group_id < group_id))].pair_id.values-1,
            data[(prev_id <= data.group_id) & (data.group_id < group_id)].pair_id.values-1)
        prev_id = group_id

In [11]:
train_data.text

0        ВАЗ 21213 | Замена подшипников ступицы | Нива ...
1         Ваз 2107 оптом в Сочи. Сравнить цены, купить ...
2        Купить ступица Лада калина2. Трансмиссия - пер...
3        Классика 21010 - 21074ГЛАВНАЯПродукцияИнтернет...
4        Ступица Нива — замена подшипника своими руками...
                               ...                        
11685                        ❤★✿★АПРЕЛЯТА 2014 -6❤★✿   ...
11686       :: Gästebuch #blockStyle404Main64 {text-ali...
11687    Jizolofej: ArchiveJizolofejMonth Filter by pos...
11688    Как зовут парня дианы шурыгиной | Пусть говоря...
11689    Культура & ШоуБиз:   Визит «Ревизорро» в Смоле...
Name: text, Length: 11690, dtype: object

In [71]:
vectorizer = TfidfVectorizer(max_features = 1000000)
texts_vectorized = vectorizer.fit_transform(train_data.text)

In [72]:
texts_vectorized.shape

(11690, 1000000)

In [73]:
for train, test in get_splits(train_data, n=3):
    print(len(train)+len(test))

11690
11690
11690


In [74]:
from sklearn.preprocessing import OneHotEncoder

In [75]:
groups = OneHotEncoder().fit_transform(train_data.group_id.values.reshape(-1, 1))

In [19]:
groups.shape

(11690, 129)

In [52]:
texts_vectorized

<11690x10000 sparse matrix of type '<class 'numpy.int64'>'
	with 12065046 stored elements in Compressed Sparse Row format>

In [76]:
from scipy.sparse import hstack

In [83]:
features = hstack( [texts_vectorized,groups] )

In [81]:
features

In [78]:
target = train_data.target.values

In [79]:
doc_id = train_data.doc_id.values

In [ ]:
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
max_score = 0
for max_depth in range(5,50):
    scores = []
    for train, test in get_splits(train_data,n=3):
                print(type(features))
                model = DecisionTreeClassifier(max_depth=None)
                model.fit(features.tocsr()[train], target[train])
                pred = model.predict(features.tocsr()[test])                
                #new_pred = np.where(pred < tresh, 0, 1)
                scores.append(f1_score(pred,target[test]))
    print(sum(scores)/3)
    if sum(scores)/3 > max_score: 
        max_score = sum(scores)/3
        max_depth_best = max_depth
                

<class 'scipy.sparse.coo.coo_matrix'>
<class 'scipy.sparse.coo.coo_matrix'>
<class 'scipy.sparse.coo.coo_matrix'>
0.4354277501688501
<class 'scipy.sparse.coo.coo_matrix'>
